In [19]:
import numpy as np
import pandas as pd
import random

In [20]:
# new data to combine with initial data (Note: formatted differently from df_2018 data)
df_ks = pd.read_csv("../Data/kickstarter.csv")

In [21]:
# Cleaning
# 1. Delete the first column (i.e., the row id 'Unnamed')
del df_ks['Unnamed: 0']

In [22]:
# 2. Replace state string with numbers in df_sk
cleanup_nums = {"state": {"failed": 0, "successful": 1, "canceled" : -1, "undefined" : -1, "live" : -1, "suspended" : -1}}
df_ks.replace(cleanup_nums, inplace=True)

In [23]:
# 3. Remove Duplicate values
# 3a. Sort by the number of backers (assuming you'll have more backers)
df_ks_sorted = df_ks.sort_values(by='backers_count')
# 3b. Remove duplicates
df_ks_sorted.drop_duplicates(subset='id', keep='last', inplace=True)

In [24]:
# 4. Create new dataframe with the columns (that are not in 2018) = 
# id, backers_count, blurb, blurb_length, spotlight, staff_pick, location_state, location_displayable_name
df_ks_dropped = df_ks_sorted[['id','backers_count', 'blurb', 'blurb_length', 'spotlight', 'staff_pick', 'location_state', 'location_displayable_name']].sort_values(by='id')

In [25]:
df_ks_dropped.head()

,id,backers_count,blurb,blurb_length,spotlight,staff_pick,location_state,location_displayable_name
343901,8624,737,A compact outdoor cooking set that you can use...,12,True,True,WV,"Belgium, WV"
212211,18520,4,Raising money to help my grandmother recover f...,24,False,False,OH,"Cleveland, OH"
11159,21109,11,My work is performance based but I branch out ...,24,True,False,Scotland,"Dundee, UK"
62717,24380,18,A sanctuary for humans and felines alike! Come...,24,False,False,IL,"Bloomington, IL"
34144,33867,42,Taste Makers is a socially conscious brand tha...,23,False,True,BC,"Vancouver, Canada"


In [26]:
# Read in other Data files
df_2018 = pd.read_csv("../Data/Cleaned/2018.csv", parse_dates= ['launched', 'deadline'])
# Rename ID to id
df_2018 = df_2018.rename(columns={"ID": "id"})

In [27]:
# Join the two tables on id
df_joined_ids = df_ks_dropped.set_index('id').join(df_2018.set_index('id'))

In [28]:
# Remove any rows that have a NaN in the name column
df_joined_ids = df_joined_ids.dropna();
df_joined_ids.head()

,backers_count,blurb,blurb_length,spotlight,staff_pick,location_state,location_displayable_name,name,category,main_category,...,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
id,,,,,,,,,,,,,,,,,,,,,
18520,4,Raising money to help my grandmother recover f...,24,False,False,OH,"Cleveland, OH",Grandma's are Life,World Music,Music,...,2016-11-18,15000.0,2016-10-19 15:32:40,62.0,0.0,4.0,US,0.00,62.00,15000.00
21109,11,My work is performance based but I branch out ...,24,True,False,Scotland,"Dundee, UK",Meta,Performance Art,Art,...,2015-05-06,150.0,2015-04-08 00:37:44,173.0,1.0,11.0,GB,258.04,266.65,231.20
24380,18,A sanctuary for humans and felines alike! Come...,24,False,False,IL,"Bloomington, IL",Puss N' Books: A relaxing cat cafe and bookstore.,Spaces,Food,...,2015-11-26,20000.0,2015-10-27 16:25:33,776.0,0.0,18.0,US,776.00,776.00,20000.00
33867,42,Taste Makers is a socially conscious brand tha...,23,False,True,BC,"Vancouver, Canada",TASTE MAKERS BY TRISH P,Ready-to-wear,Fashion,...,2015-07-30,18000.0,2015-06-15 20:28:11,2798.0,0.0,42.0,CA,2271.72,2144.39,13795.22
39036,68,"The BEST beef sticks, beef jerky and signature...",13,True,False,WI,"Kronenwetter, WI",The Meat Candy Experience,Small Batch,Food,...,2016-07-01,2500.0,2016-05-17 00:34:18,3239.0,1.0,68.0,US,3239.00,3239.00,2500.00


In [29]:
# Convert Timestamps
df_joined_ids = df_joined_ids.assign(
                launch_hour=df_joined_ids.launched.dt.hour,
                launch_day=df_joined_ids.launched.dt.day,
                launch_month=df_joined_ids.launched.dt.month,
                launch_year=df_joined_ids.launched.dt.year)

df_joined_ids = df_joined_ids.assign(
                deadline_day=df_joined_ids.deadline.dt.day,
                deadline_month=df_joined_ids.deadline.dt.month,
                deadline_year=df_joined_ids.deadline.dt.year)

In [30]:
# Add column: project duration in days
#from datetime import datetime
df_joined_ids['launched'] = pd.to_datetime(df_joined_ids['launched']).dt.date
df_joined_ids['deadline'] = pd.to_datetime(df_joined_ids['deadline']).dt.date

df_joined_ids['duration_days'] = (df_joined_ids['deadline'] - df_joined_ids['launched']).dt.days

In [31]:
df_joined_ids.head(3)

,backers_count,blurb,blurb_length,spotlight,staff_pick,location_state,location_displayable_name,name,category,main_category,...,usd_pledged_real,usd_goal_real,launch_hour,launch_day,launch_month,launch_year,deadline_day,deadline_month,deadline_year,duration_days
id,,,,,,,,,,,,,,,,,,,,,
18520,4,Raising money to help my grandmother recover f...,24,False,False,OH,"Cleveland, OH",Grandma's are Life,World Music,Music,...,62.00,15000.0,15,19,10,2016,18,11,2016,30
21109,11,My work is performance based but I branch out ...,24,True,False,Scotland,"Dundee, UK",Meta,Performance Art,Art,...,266.65,231.2,0,8,4,2015,6,5,2015,28
24380,18,A sanctuary for humans and felines alike! Come...,24,False,False,IL,"Bloomington, IL",Puss N' Books: A relaxing cat cafe and bookstore.,Spaces,Food,...,776.00,20000.0,16,27,10,2015,26,11,2015,30


In [32]:
# Drop Redundant Columns
df_joined_ids = df_joined_ids.drop(columns=['location_displayable_name'])
df_joined_ids = df_joined_ids.drop(columns=['backers'])
df_joined_ids = df_joined_ids.drop(columns=['usd pledged'])
df_joined_ids = df_joined_ids.drop(columns=['currency'])
df_joined_ids = df_joined_ids.drop(columns=['pledged'])
df_joined_ids = df_joined_ids.drop(columns=['goal'])
df_joined_ids = df_joined_ids.drop(columns=['launched'])
df_joined_ids = df_joined_ids.drop(columns=['deadline'])

In [33]:
df_joined_ids.head()

,backers_count,blurb,blurb_length,spotlight,staff_pick,location_state,name,category,main_category,state,...,usd_pledged_real,usd_goal_real,launch_hour,launch_day,launch_month,launch_year,deadline_day,deadline_month,deadline_year,duration_days
id,,,,,,,,,,,,,,,,,,,,,
18520,4,Raising money to help my grandmother recover f...,24,False,False,OH,Grandma's are Life,World Music,Music,0.0,...,62.00,15000.00,15,19,10,2016,18,11,2016,30
21109,11,My work is performance based but I branch out ...,24,True,False,Scotland,Meta,Performance Art,Art,1.0,...,266.65,231.20,0,8,4,2015,6,5,2015,28
24380,18,A sanctuary for humans and felines alike! Come...,24,False,False,IL,Puss N' Books: A relaxing cat cafe and bookstore.,Spaces,Food,0.0,...,776.00,20000.00,16,27,10,2015,26,11,2015,30
33867,42,Taste Makers is a socially conscious brand tha...,23,False,True,BC,TASTE MAKERS BY TRISH P,Ready-to-wear,Fashion,0.0,...,2144.39,13795.22,20,15,6,2015,30,7,2015,45
39036,68,"The BEST beef sticks, beef jerky and signature...",13,True,False,WI,The Meat Candy Experience,Small Batch,Food,1.0,...,3239.00,2500.00,0,17,5,2016,1,7,2016,45


In [34]:
df_joined_ids.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 242630 entries, 18520 to 2147476221
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   backers_count     242630 non-null  int64  
 1   blurb             242630 non-null  object 
 2   blurb_length      242630 non-null  int64  
 3   spotlight         242630 non-null  bool   
 4   staff_pick        242630 non-null  bool   
 5   location_state    242630 non-null  object 
 6   name              242630 non-null  object 
 7   category          242630 non-null  object 
 8   main_category     242630 non-null  object 
 9   state             242630 non-null  float64
 10  country           242630 non-null  object 
 11  usd_pledged_real  242630 non-null  float64
 12  usd_goal_real     242630 non-null  float64
 13  launch_hour       242630 non-null  int64  
 14  launch_day        242630 non-null  int64  
 15  launch_month      242630 non-null  int64  
 16  launch_year 

In [35]:
df_joined_ids.to_csv("../Data/Cleaned/joined_data.csv", index = False)